# **Fine-tuned LLM Benchmarking Notebook 1**

In [1]:
import os, warnings, torch, gc
import pandas as pd
from tqdm.notebook import tqdm
from transformers import (
    AutoTokenizer, 
    LlamaTokenizer,
    AutoModelForCausalLM,
    LlamaForCausalLM,
    AutoModelForSeq2SeqLM, 
    AutoConfig,
    pipeline
)
from datasets import load_dataset
from datasets.dataset_dict import DatasetDict
from peft import PeftConfig, PeftModel
from rouge import Rouge
from sentence_transformers import SentenceTransformer
from langchain.llms import HuggingFacePipeline
from langchain.chains.summarize import load_summarize_chain
from langchain.schema import Document
from sklearn.metrics.pairwise import cosine_similarity
from typing import Dict, Iterable, Union, Any, Optional

warnings.filterwarnings("ignore")

os.environ["EMBEDDINGS_MODEL"] = "all-MiniLM-L12-v2"
os.environ["MAX_TOKENS"] = "2048"
os.environ["DEVICE"] = "cuda:0" if torch.cuda.is_available() else "cpu"
os.environ["DATASET_PATH"] = "data/doc_summary_data"
os.environ["FINETUNED_FLAN_T5_SMALL_ID"] = "flan-t5-small_finetuned_results"
os.environ["FINETUNED_FLAN_T5_BASE_ID"] = "flan-t5-base_finetuned_results"
os.environ["ALPACA_LLM"] = "chavinlo/alpaca-native"
os.environ["VICUNA_LLM"] = "lmsys/vicuna-13b-v1.3"

In [2]:
def run_on_test_data(
    model: Union[AutoModelForSeq2SeqLM, AutoModelForCausalLM, LlamaForCausalLM], 
    tokenizer: Union[AutoTokenizer, LlamaTokenizer], 
    dataset_key: str="test",
    task="text2text-generation",
    n_docs: int = 5,
    log_summary: bool=False,
    log_metrics: bool=False,
    **kwargs):
    
    # switch model to eval mode
    model.eval()
    
    # Define model pipeline for inference with langchain
    kwargs = {**dict(temperature=0.1, top_p=0.15, top_k=0, repetition_penalty=1.1), **kwargs}

    # define model pipeline
    hgf_pipeline = pipeline(
        task=task, 
        model=model, 
        tokenizer=tokenizer,
        max_length=int(os.environ["MAX_TOKENS"]),
        **kwargs
    )
    llm = HuggingFacePipeline(pipeline=hgf_pipeline)
    
    # Define Summary chain
    summary_chain = load_summarize_chain(llm, chain_type="map_reduce")
    
    # Load dataset
    dataset: DatasetDict = load_dataset(path=os.environ["DATASET_PATH"])
    
    # Generate Summaries and Measure Performance (Rouge Metric and Cosine Similarity Metric)
    rouge = Rouge()  # rouge metric object
    embeddings_model = SentenceTransformer(os.environ["EMBEDDINGS_MODEL"]) # embeddings model object
    embeddings_model.to(os.environ["DEVICE"])
    documents = dataset[dataset_key]["document"][:n_docs]
    target_summaries = dataset[dataset_key]["summary"][:n_docs]
    _zipped = zip(documents, target_summaries)
    metrics_values: Iterable[Dict[str, Any]] = []

    for i, (document, target_summary) in enumerate(_zipped):
        document = Document(page_content=document)
        try:
            generated_summary = summary_chain.run([document])
        except ValueError as e:
            print(f"Error summarizing document-{i+1}: {e}")
            continue
            
        if log_summary:
            print(f"GENERATED SUMARY: {generated_summary}\n")
            print(f"TARGET SUMARY: {target_summary}\n")

        generated_summary_embeddings, target_summary_embeddings = (
            embeddings_model.encode(generated_summary).reshape(1, -1),
            embeddings_model.encode(target_summary).reshape(1, -1)
        )
        cos_similarity = cosine_similarity(target_summary_embeddings, generated_summary_embeddings)
        rouge_scores = rouge.get_scores(generated_summary, target_summary)
        if log_metrics:
            print(f"Cosine similarity for summary {i+1}:", cos_similarity[0][0], "\n")
            print(f"Rouge scores for summary {i+1}:", rouge_scores[0], "\n")
            
        if log_metrics or log_summary:
            print("\n")
        _metric = dict(semantic_similarity=cos_similarity, rouge_scores=rouge_scores)
        metrics_values.append(_metric)
        
    return metrics_values

In [3]:
INFERENCE_DATASET_KEY = "test"
N_INFERENCE_DOCS = 5
LOG_SUMMARY = True
LOG_METRICS = True
DELETE_LLM_AFTER_USE = True

## Evaluate Base Pre-trained FLAN-T5-SMALL

In [4]:
# Load peft config for pre-trained checkpoint etc. 
flan_t5_small_config = PeftConfig.from_pretrained(os.environ["FINETUNED_FLAN_T5_SMALL_ID"])

# load base LLM model and tokenizer
flan_t5_small_model = AutoModelForSeq2SeqLM.from_pretrained(
    flan_t5_small_config.base_model_name_or_path,  
    load_in_8bit=True,  
    device_map="auto",
)
flan_t5_small_tokenizer = AutoTokenizer.from_pretrained(
    flan_t5_small_config.base_model_name_or_path, 
)

flan_t5_small_performance = run_on_test_data(
    flan_t5_small_model, 
    flan_t5_small_tokenizer, 
    INFERENCE_DATASET_KEY, 
    task="text2text-generation", 
    n_docs=N_INFERENCE_DOCS,
    log_summary=LOG_SUMMARY,
    log_metrics=LOG_METRICS,
)

Token indices sequence length is longer than the specified maximum sequence length for this model (1932 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2066 > 512). Running this sequence through the model will result in indexing errors


GENERATED SUMARY: Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United States, Miami Cancer Institute in Orlando, Florida, 33028, United 

Token indices sequence length is longer than the specified maximum sequence length for this model (1802 > 1024). Running this sequence through the model will result in indexing errors


GENERATED SUMARY: The Cancer Center of Hawaii-Liliha, Honolulu, Hawaii, 96813, United States, United States, Canada, United States, Canada, United States, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canada, United Kingdom, Canad

Token indices sequence length is longer than the specified maximum sequence length for this model (1981 > 1024). Running this sequence through the model will result in indexing errors


GENERATED SUMARY: United States, United States, and the University of Michigan Comprehensive Cancer Center, Ann Arbor, Michigan, 48108, United States, United States, and the University of Michigan Comprehensive Cancer Center, Kalamazoo, Michigan, 48342, United States, and the University of Michigan Comprehensive Cancer Center, Kalamazoo, Michigan, 48342, United States, and the University of Michigan Comprehensive Cancer Center, Kalamazoo, Michigan, 48342, United States, and the University of Michigan Comprehensive Cancer Center, Kalamazoo, Michigan, 48342, United States, and the University of Michigan Comprehensive Cancer Center, Kalamazoo, Michigan, 48342, United States, and the University of Michigan Comprehensive Cancer Center, Kalamazoo, Michigan, 48342, United States, and the University of Michigan Comprehensive Cancer Center, Kalamazoo, Michigan, 48342, United States, and the University of Michigan Comprehensive Cancer Center, Kalamazoo, Michigan, 48342, United States, and the Un

## Evaluate LoRA Fine-tuned FLAN-T5-SMALL

In [5]:
# load peft flan-t5-small
flan_t5_small_model = PeftModel.from_pretrained(
    flan_t5_small_model, 
    os.environ["FINETUNED_FLAN_T5_SMALL_ID"],
    device_map="auto"
)

lora_flan_t5_small_performance = run_on_test_data(
    flan_t5_small_model, 
    flan_t5_small_tokenizer, 
    INFERENCE_DATASET_KEY, 
    task="text2text-generation", 
    n_docs=N_INFERENCE_DOCS,
    log_summary=LOG_SUMMARY,
    log_metrics=LOG_METRICS,
)

if DELETE_LLM_AFTER_USE:
    #model.to(torch.device("cpu"))
    del flan_t5_small_model, flan_t5_small_tokenizer
    gc.collect()
    torch.cuda.empty_cache()

The model 'PeftModelForSeq2SeqLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].
Token indices sequence length is longer than the specified maximum sequence length for this model (1938 > 1024

GENERATED SUMARY: The summary provides a list of cancer centers and hospitals in different cities in Florida, including Jacksonville, Delaware, and Newnan.

TARGET SUMARY: The summary provides a list of cancer centers and hospitals in various states in the US, such as Delaware, Florida, and Georgia, that provide cancer treatment and care services.

Cosine similarity for summary 1: 0.8069508 

Rouge scores for summary 1: {'rouge-1': {'r': 0.52, 'p': 0.7222222222222222, 'f': 0.6046511579232018}, 'rouge-2': {'r': 0.35714285714285715, 'p': 0.5263157894736842, 'f': 0.42553191007695795}, 'rouge-l': {'r': 0.48, 'p': 0.6666666666666666, 'f': 0.5581395300162251}} 



GENERATED SUMARY: This study includes medical centers and hospitals in different cities in the United States, including Georgia, Hawaii, and Illinois.

TARGET SUMARY: The summary provides a list of medical centers and hospitals across multiple states in the US that specialize in oncology and cancer treatment.

Cosine similarity for

## Evaluate Base Pretrained FLAN-T5-BASE

In [6]:
# Load peft config for pre-trained checkpoint etc. 
flan_t5_base_config = PeftConfig.from_pretrained(os.environ["FINETUNED_FLAN_T5_BASE_ID"])

# load base LLM model and tokenizer
flan_t5_base_model = AutoModelForSeq2SeqLM.from_pretrained(
    flan_t5_base_config.base_model_name_or_path,  
    load_in_8bit=True,  
    device_map="auto",
)

flan_t5_base_tokenizer = AutoTokenizer.from_pretrained(
    flan_t5_base_config.base_model_name_or_path, 
)

flan_t5_base_performance = run_on_test_data(
    flan_t5_base_model, 
    flan_t5_base_tokenizer, 
    INFERENCE_DATASET_KEY, 
    task="text2text-generation", 
    n_docs=N_INFERENCE_DOCS,
    log_summary=LOG_SUMMARY,
    log_metrics=LOG_METRICS,
)

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GENERATED SUMARY: The following is a list of cancer centers in the United States:

TARGET SUMARY: The summary provides a list of cancer centers and hospitals in various states in the US, such as Delaware, Florida, and Georgia, that provide cancer treatment and care services.

Cosine similarity for summary 1: 0.827664 

Rouge scores for summary 1: {'rouge-1': {'r': 0.32, 'p': 0.6666666666666666, 'f': 0.4324324280496713}, 'rouge-2': {'r': 0.17857142857142858, 'p': 0.45454545454545453, 'f': 0.25641025236028936}, 'rouge-l': {'r': 0.32, 'p': 0.6666666666666666, 'f': 0.4324324280496713}} 



GENERATED SUMARY: The following hospitals in Newnan, Georgia are listed below:

TARGET SUMARY: The summary provides a list of medical centers and hospitals across multiple states in the US that specialize in oncology and cancer treatment.

Cosine similarity for summary 2: 0.48196936 

Rouge scores for summary 2: {'rouge-1': {'r': 0.14285714285714285, 'p': 0.3333333333333333, 'f': 0.19999999580000008}, 'r

Token indices sequence length is longer than the specified maximum sequence length for this model (1876 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2068 > 512). Running this sequence through the model will result in indexing errors


GENERATED SUMARY: Maine Health/SMHC Cancer Care and Blood Disorders-Biddeford, Biddeford, Maine, 04005, United States|Maine Medical Center-Bramhall Campus, Portland, Maine, 04102, United States|Maine Medical Center-Bramhall Campus, Scarborough, Maine, 04074, United States|University of Maryland/Greenebaum Cancer Center, Baltimore, Maryland, 21201, United States|Greater Baltimore Medical Center, Baltimore, Maryland, 21204, United States|UM Upper Chesapeake Medical Center, Baltimore, Maryland, 21204, United States|UM Upper Chesapeake Medical Center, Bel Air, Maryland, 21044, United States|Lahey Hospital and Medical Center, Burlington, Massachusetts, 01805, United States|Lahey Hospital and Medical Center, Burlington, Massachusetts, 01805, United States|Lahey Hospital and Medical Center, Burlington, Massachusetts, 01805, United States|Lahey Hospital and Medical Center, Burlington, Massachusetts, 01805, United States|Lahey Hospital and Medical Center, Burlington, Massachusetts, 01805, Unite

## Evaluate LoRA Fine-tuned FLAN-T5-BASE

In [7]:
# Load peft flan-t5-base
flan_t5_base_model = PeftModel.from_pretrained(
    flan_t5_base_model, 
    os.environ["FINETUNED_FLAN_T5_BASE_ID"],
    device_map="auto"
)
flan_t5_base_tokenizer = AutoTokenizer.from_pretrained(
    flan_t5_base_config.base_model_name_or_path, 
    max_length=os.environ["MAX_TOKENS"],
)

lora_flan_t5_base_performance = run_on_test_data(
    flan_t5_base_model, 
    flan_t5_base_tokenizer, 
    INFERENCE_DATASET_KEY, 
    task="text2text-generation", 
    n_docs=N_INFERENCE_DOCS,
    log_summary=LOG_SUMMARY,
    log_metrics=LOG_METRICS,
)

if DELETE_LLM_AFTER_USE:
    #model.to(torch.device("cpu"))
    del flan_t5_base_model, flan_t5_base_tokenizer
    gc.collect()
    torch.cuda.empty_cache()

The model 'PeftModelForSeq2SeqLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


GENERATED SUMARY: The summary provides a list of cancer centers and hospitals in different states in the US, including Delaware, Florida, and Georgia.

TARGET SUMARY: The summary provides a list of cancer centers and hospitals in various states in the US, such as Delaware, Florida, and Georgia, that provide cancer treatment and care services.

Cosine similarity for summary 1: 0.95705646 

Rouge scores for summary 1: {'rouge-1': {'r': 0.64, 'p': 0.8421052631578947, 'f': 0.7272727223657025}, 'rouge-2': {'r': 0.5357142857142857, 'p': 0.75, 'f': 0.624999995138889}, 'rouge-l': {'r': 0.64, 'p': 0.8421052631578947, 'f': 0.7272727223657025}} 



GENERATED SUMARY: The summary provides a list of medical centers and hospitals in various cities across the United States, including Georgia, Hawaii, and Illinois.

TARGET SUMARY: The summary provides a list of medical centers and hospitals across multiple states in the US that specialize in oncology and cancer treatment.

Cosine similarity for summary

## Evaluate Base Pretrained Alpaca Native

In [8]:
alpaca_config = AutoConfig.from_pretrained(os.environ["ALPACA_LLM"])

alpaca_model = LlamaForCausalLM.from_pretrained(
    os.environ["ALPACA_LLM"],
    load_in_8bit=True,
    device_map="auto",
)

alpaca_tokenizer = LlamaTokenizer.from_pretrained(
    os.environ["ALPACA_LLM"], 
    max_length=alpaca_config.max_sequence_length,
)

alpaca_native_performance = run_on_test_data(
    alpaca_model, 
    alpaca_tokenizer, 
    INFERENCE_DATASET_KEY, 
    task="text-generation", 
    n_docs=N_INFERENCE_DOCS,
    log_summary=LOG_SUMMARY,
    log_metrics=LOG_METRICS,
)

if DELETE_LLM_AFTER_USE:
    #model.to(torch.device("cpu"))
    del alpaca_model, alpaca_tokenizer
    gc.collect()
    torch.cuda.empty_cache()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


GENERATED SUMARY: 

The Helen F. Graham Cancer Center is located in Newark, Delaware and provides comprehensive cancer care services. The University of Florida Health Science Center is located in Gainesville, Florida and offers a range of specialized cancer treatments and clinical trials. Memorial Regional Hospital/Joe DiMaggio Children's Hospital is located in Hollywood, Florida and provides pediatric oncology services. The Mayo Clinic in Florida is located in Jacksonville, Florida and specializes in medical and surgical oncology. The Miami Cancer Institute is located in Miami, Florida and provides advanced treatment options for cancer patients. The Orlando Health Cancer Institute is located in Orlando, Florida and provides comprehensive cancer care services. Memorial Hospital West is located in Pembroke Pines, Florida and provides radiation therapy and chemotherapy services. Emory University Hospital Midtown is located in Atlanta, Georgia and provides a range of specialized cancer tr

## Evaluate Base Pretrained Vicuna 13b v1.3

In [9]:
vicuna_model = AutoModelForCausalLM.from_pretrained(
    os.environ["VICUNA_LLM"],
    load_in_8bit=True,
    device_map="auto",
)

vicuna_tokenizer = AutoTokenizer.from_pretrained(
    os.environ["VICUNA_LLM"], 
    max_length=os.environ["MAX_TOKENS"],
)

vicuna_13b_performance = run_on_test_data(
    vicuna_model, 
    vicuna_tokenizer, 
    INFERENCE_DATASET_KEY, 
    task="text-generation", 
    n_docs=N_INFERENCE_DOCS,
    log_summary=LOG_SUMMARY,
    log_metrics=LOG_METRICS,
)

if DELETE_LLM_AFTER_USE:
    #model.to(torch.device("cpu"))
    del vicuna_model, vicuna_tokenizer
    gc.collect()
    torch.cuda.empty_cache()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GENERATED SUMARY: 

The text provides a list of various cancer centers and hospitals across the United States, including locations in Delaware, Florida, Georgia, and Atlanta. Some of these institutions are affiliated with universities or have multiple campuses, such as the Mayo Clinic and Emory University Hospital.

TARGET SUMARY: The summary provides a list of cancer centers and hospitals in various states in the US, such as Delaware, Florida, and Georgia, that provide cancer treatment and care services.

Cosine similarity for summary 1: 0.8590759 

Rouge scores for summary 1: {'rouge-1': {'r': 0.68, 'p': 0.425, 'f': 0.5230769183431954}, 'rouge-2': {'r': 0.2857142857142857, 'p': 0.18604651162790697, 'f': 0.22535210789922644}, 'rouge-l': {'r': 0.6, 'p': 0.375, 'f': 0.4615384568047337}} 



GENERATED SUMARY: 
The passage provides a list of various medical centers and hospitals located throughout the United States, including Southeastern Regional Medical Center in Newnan, Georgia; Lewis 

In [10]:
def compile_metrics(performance: Iterable[Dict[str, Any]], outer_key: Optional[str]=None) -> Dict[str, Iterable]:
    performance_table_dict = dict()
    performance_dict = dict()
    join_keys = lambda inner_key : (outer_key, inner_key) if outer_key else inner_key
    
    for i, item in enumerate(performance):
        similarity_score = item["semantic_similarity"][0][0]
        rouge_scores = item["rouge_scores"][0]
        if i == 0:
            performance_dict[join_keys("similarity_score")]=[similarity_score]
        else:
            performance_dict[join_keys("similarity_score")].append(similarity_score)
        for key in rouge_scores.keys():
            if i == 0:
                performance_dict = {
                    **performance_dict, 
                    **{join_keys(f"{key}_{k}"):[v] for k, v in rouge_scores[key].items()}
                }
            else:
                for k, v in rouge_scores[key].items():
                    performance_dict[join_keys(f"{key}_{k}")].append(v)
    return performance_dict

In [11]:
performances = {
    "flan-t5-small": flan_t5_small_performance,
    "lora-peft-flan-t5-small": lora_flan_t5_small_performance,
    "flan-t5-base": flan_t5_base_performance,
    "lora-peft-flan-t5-base": lora_flan_t5_base_performance,
    "alpaca-7b-native": alpaca_native_performance,
    "vicuna-13b-v1.3": vicuna_13b_performance,
}
metrics_dict = dict()
for key, performance_values in performances.items():
    metrics_dict = {**metrics_dict, **compile_metrics(performance_values, outer_key=key)}
    
outer_cols = [k for k in performances.keys()]
inner_cols = [k[1] for k in metrics_dict.keys() if k[0]=="flan-t5-small"]
columns = pd.MultiIndex.from_product([outer_cols, inner_cols])
metrics_df = pd.DataFrame(metrics_dict, columns=columns).round(2)

In [12]:
pd.set_option('display.max_columns', None)
metrics_df

flan-t5-small                                                    \
  similarity_score rouge-1_r rouge-1_p rouge-1_f rouge-2_r rouge-2_p   
0             0.60      0.08      0.20      0.11      0.00      0.00   
1             0.53      0.10      0.17      0.12      0.00      0.00   
2             0.53      0.05      0.02      0.03      0.00      0.00   
3             0.43      0.17      0.36      0.24      0.04      0.07   
4             0.74      0.21      0.19      0.20      0.00      0.00   

                                          lora-peft-flan-t5-small            \
  rouge-2_f rouge-l_r rouge-l_p rouge-l_f        similarity_score rouge-1_r   
0      0.00      0.08      0.20      0.11                    0.81      0.52   
1      0.00      0.10      0.17      0.12                    0.60      0.29   
2      0.00      0.05      0.02      0.03                    0.70      0.45   
3      0.05      0.13      0.27      0.18                    0.80      0.61   
4      0.00      0.21      0.19      0.20                    0.85      0.79   

                                                                         \
  rouge-1_p rouge-1_f rouge-2_r rouge-2_p rouge-2_f rouge-l_r rouge-l_p   
0      0.72      0.60      0.36      0.53      0.43      0.48      0.67   
1      0.35      0.32      0.18      0.22      0.20      0.24      0.29   
2      0.60      0.51      0.26      0.31      0.29      0.45      0.60   
3      0.78      0.68      0.50      0.63      0.56      0.61      0.78   
4      0.65      0.71      0.64      0.53      0.58      0.79      0.65   

                flan-t5-base                                          \
  rouge-l_f similarity_score rouge-1_r rouge-1_p rouge-1_f rouge-2_r   
0      0.56             0.83      0.32      0.67      0.43      0.18   
1      0.26             0.48      0.14      0.33      0.20      0.00   
2      0.51             0.60      0.30      0.60      0.40      0.11   
3      0.68             0.58      0.09      0.05      0.06      0.00   
4      0.71             0.74      0.50      0.58      0.54      0.29   

                                                    lora-peft-flan-t5-base  \
  rouge-2_p rouge-2_f rouge-l_r rouge-l_p rouge-l_f       similarity_score   
0      0.45      0.26      0.32      0.67      0.43                   0.96   
1      0.00      0.00      0.14      0.33      0.20                   0.72   
2      0.22      0.14      0.25      0.50      0.33                   0.82   
3      0.00      0.00      0.09      0.05      0.06                   0.82   
4      0.36      0.32      0.50      0.58      0.54                   0.84   

                                                                         \
  rouge-1_r rouge-1_p rouge-1_f rouge-2_r rouge-2_p rouge-2_f rouge-l_r   
0      0.64      0.84      0.73      0.54      0.75      0.62      0.64   
1      0.62      0.62      0.62      0.41      0.43      0.42      0.57   
2      0.45      0.43      0.44      0.26      0.23      0.24      0.45   
3      0.57      0.68      0.62      0.46      0.55      0.50      0.57   
4      0.79      0.85      0.81      0.64      0.69      0.67      0.79   

                      alpaca-7b-native                                \
  rouge-l_p rouge-l_f similarity_score rouge-1_r rouge-1_p rouge-1_f   
0      0.84      0.73             0.68      0.40      0.12      0.19   
1      0.57      0.57             0.71      0.05      0.02      0.03   
2      0.43      0.44             0.68      0.20      0.22      0.21   
3      0.68      0.62             0.72      0.39      0.27      0.32   
4      0.85      0.81             0.69      0.43      0.10      0.17   

                                                               \
  rouge-2_r rouge-2_p rouge-2_f rouge-l_r rouge-l_p rouge-l_f   
0      0.07      0.02      0.03      0.40      0.12      0.19   
1      0.00      0.00      0.00      0.05      0.02      0.03   
2      0.00      0.00      0.00      0.15      0.17      0.16   
3      0.08 